In [143]:
#!/usr/bin/python
%matplotlib inline

import os
import sys
sys.path.append("../../tools/")
import pickle
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
from sklearn.cross_validation import train_test_split, StratifiedShuffleSplit
import matplotlib.pyplot as plt
from operator import itemgetter
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.pipeline import FeatureUnion
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import tester
import numpy as np
import pandas as pd
from time import time

In [144]:
# Load data and store as pandas dataframe

with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

# Remove outlier
data_dict.pop('TOTAL', 0)
data_dict.pop('THE TRAVEL AGENCY IN THE PARK', 0)

{'bonus': 'NaN',
 'deferral_payments': 'NaN',
 'deferred_income': 'NaN',
 'director_fees': 'NaN',
 'email_address': 'NaN',
 'exercised_stock_options': 'NaN',
 'expenses': 'NaN',
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 362096,
 'poi': False,
 'restricted_stock': 'NaN',
 'restricted_stock_deferred': 'NaN',
 'salary': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 362096,
 'total_stock_value': 'NaN'}

In [145]:
df = pd.DataFrame(data_dict).T
#df = df.apply(pd.to_numeric)
print df.shape

(144, 21)


In [146]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 144 entries, ALLEN PHILLIP K to YEAP SOON
Data columns (total 21 columns):
bonus                        144 non-null object
deferral_payments            144 non-null object
deferred_income              144 non-null object
director_fees                144 non-null object
email_address                144 non-null object
exercised_stock_options      144 non-null object
expenses                     144 non-null object
from_messages                144 non-null object
from_poi_to_this_person      144 non-null object
from_this_person_to_poi      144 non-null object
loan_advances                144 non-null object
long_term_incentive          144 non-null object
other                        144 non-null object
poi                          144 non-null object
restricted_stock             144 non-null object
restricted_stock_deferred    144 non-null object
salary                       144 non-null object
shared_receipt_with_poi      144 non-null objec

In [147]:
df['bonus'] = df['bonus'].astype('float64')
df['deferral_payments'] = df['deferral_payments'].astype('float64')
df['deferred_income'] = df['deferred_income'].astype('float64')
df['director_fees'] = df['director_fees'].astype('float64')
df['email_address'] = df['bonus'].astype('str')
df['exercised_stock_options'] = df['exercised_stock_options'].astype('float64')
df['expenses'] = df['expenses'].astype('float64')
df['from_messages'] = df['from_messages'].astype('float64')
df['from_poi_to_this_person'] = df['from_poi_to_this_person'].astype('float64')
df['from_this_person_to_poi'] = df['from_this_person_to_poi'].astype('float64')
df['loan_advances'] = df['loan_advances'].astype('float64')
df['long_term_incentive'] = df['long_term_incentive'].astype('float64')
df['other'] = df['other'].astype('float64')
df['poi'] = df['poi'].astype('bool')
df['restricted_stock'] = df['restricted_stock'].astype('float64')
df['restricted_stock_deferred'] = df['restricted_stock_deferred'].astype('float64')
df['salary'] = df['salary'].astype('float64')
df['shared_receipt_with_poi'] = df['shared_receipt_with_poi'].astype('float64')
df['to_messages'] = df['to_messages'].astype('float64')
df['total_payments'] = df['total_payments'].astype('float64')
df['total_stock_value'] = df['total_stock_value'].astype('float64')

In [148]:
print df.dtypes

bonus                        float64
deferral_payments            float64
deferred_income              float64
director_fees                float64
email_address                 object
exercised_stock_options      float64
expenses                     float64
from_messages                float64
from_poi_to_this_person      float64
from_this_person_to_poi      float64
loan_advances                float64
long_term_incentive          float64
other                        float64
poi                             bool
restricted_stock             float64
restricted_stock_deferred    float64
salary                       float64
shared_receipt_with_poi      float64
to_messages                  float64
total_payments               float64
total_stock_value            float64
dtype: object


In [149]:
df.describe()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
count,8.100000e+01,3.800000e+01,4.800000e+01,16.000000,1.010000e+02,94.000000,86.000000,86.000000,86.000000,3.000000e+00,6.500000e+01,9.100000e+01,1.090000e+02,1.700000e+01,9.400000e+01,86.000000,86.000000,1.230000e+02,1.250000e+02
mean,1.201773e+06,8.416025e+05,-5.810498e+05,89822.875000,2.959559e+06,54192.010638,608.790698,64.895349,41.232558,2.797500e+07,7.464912e+05,4.664105e+05,1.147424e+06,6.218928e+05,2.840875e+05,1176.465116,2073.860465,2.641806e+06,3.352073e+06
std,1.441679e+06,1.289323e+06,9.420764e+05,41112.700735,5.499450e+06,46108.377454,1841.033949,86.979244,100.073111,4.638256e+07,8.629174e+05,1.397376e+06,2.249770e+06,3.845528e+06,1.771311e+05,1178.317641,2582.700981,9.524694e+06,6.532883e+06
min,7.000000e+04,-1.025000e+05,-3.504386e+06,3285.000000,3.285000e+03,148.000000,12.000000,0.000000,0.000000,4.000000e+05,6.922300e+04,2.000000e+00,-2.604490e+06,-1.787380e+06,4.770000e+02,2.000000,57.000000,1.480000e+02,-4.409300e+04
25%,4.250000e+05,7.964450e+04,-6.112092e+05,83674.500000,5.067650e+05,22479.000000,22.750000,10.000000,1.000000,1.200000e+06,2.750000e+05,1.203000e+03,2.520550e+05,-3.298250e+05,2.118020e+05,249.750000,541.250000,3.969340e+05,4.941360e+05
50%,7.500000e+05,2.210635e+05,-1.519270e+05,106164.500000,1.297049e+06,46547.500000,41.000000,35.000000,8.000000,2.000000e+06,4.221580e+05,5.158700e+04,4.410960e+05,-1.402640e+05,2.587410e+05,740.500000,1211.000000,1.101393e+06,1.095040e+06
75%,1.200000e+06,8.672112e+05,-3.792600e+04,112815.000000,2.542813e+06,78408.500000,145.500000,72.250000,24.750000,4.176250e+07,8.318090e+05,3.319830e+05,9.850320e+05,-7.241900e+04,3.086065e+05,1888.250000,2634.750000,2.087530e+06,2.606763e+06
max,8.000000e+06,6.426990e+06,-8.330000e+02,137864.000000,3.434838e+07,228763.000000,14368.000000,528.000000,609.000000,8.152500e+07,5.145434e+06,1.035973e+07,1.476169e+07,1.545629e+07,1.111258e+06,5521.000000,15149.000000,1.035598e+08,4.911008e+07


In [150]:
1-df.count()/144

bonus                        0.437500
deferral_payments            0.736111
deferred_income              0.666667
director_fees                0.888889
email_address                0.000000
exercised_stock_options      0.298611
expenses                     0.347222
from_messages                0.402778
from_poi_to_this_person      0.402778
from_this_person_to_poi      0.402778
loan_advances                0.979167
long_term_incentive          0.548611
other                        0.368056
poi                          0.000000
restricted_stock             0.243056
restricted_stock_deferred    0.881944
salary                       0.347222
shared_receipt_with_poi      0.402778
to_messages                  0.402778
total_payments               0.145833
total_stock_value            0.131944
dtype: float64

In [151]:
payment_fields = ['salary',
                  'bonus', 
                  'long_term_incentive', 
                  'deferred_income',
                  'deferral_payments',
                  'loan_advances',
                  'other',
                  'expenses',
                  'director_fees',
                  'total_payments']

stock_fields = ['exercised_stock_options',
                'restricted_stock',
                'restricted_stock_deferred',
                'total_stock_value']

email_data = ['from_messages',
              'from_poi_to_this_person',
              'from_this_person_to_poi',
              'shared_receipt_with_poi',
              'to_messages']
df[payment_fields] = df[payment_fields].fillna(0)
df[stock_fields] = df[stock_fields].fillna(0)

from sklearn.preprocessing import Imputer

imp = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
df_poi = df[df['poi'] == True]
df_nonpoi = df[df['poi'] == False]

df_poi.ix[:,email_data] = imp.fit_transform(df_poi.ix[:,email_data]);
df_nonpoi.ix[:,email_data] = imp.fit_transform(df_nonpoi.ix[:,email_data]);

df = df_poi.append(df_nonpoi)

In [152]:
1-df.count()/144

bonus                        0.0
deferral_payments            0.0
deferred_income              0.0
director_fees                0.0
email_address                0.0
exercised_stock_options      0.0
expenses                     0.0
from_messages                0.0
from_poi_to_this_person      0.0
from_this_person_to_poi      0.0
loan_advances                0.0
long_term_incentive          0.0
other                        0.0
poi                          0.0
restricted_stock             0.0
restricted_stock_deferred    0.0
salary                       0.0
shared_receipt_with_poi      0.0
to_messages                  0.0
total_payments               0.0
total_stock_value            0.0
dtype: float64

In [153]:
# First include all relevent features
features_list = ['poi',
                 'bonus',
                 'salary',
                 'deferral_payments',
                 'deferred_income',
                 'director_fees',
                 'exercised_stock_options',
                 'expenses',
                 'from_messages',
                 'from_poi_to_this_person',
                 'from_this_person_to_poi',
                 'loan_advances',
                 'long_term_incentive',
                 'other', 
                 'restricted_stock',
                 'restricted_stock_deferred', 
                 'shared_receipt_with_poi',
                 'to_messages', 
                 'total_payments', 
                 'total_stock_value']

In [154]:
# Seperate labels and features
data_dict_cleaned = df.to_dict(orient = 'index')
my_dataset = df.to_dict(orient = 'index')
#my_dataset = data_dict
data = featureFormat(my_dataset, features_list)
labels, features = targetFeatureSplit(data)

In [155]:
# We use StratifiedShuffleSplit because the dataset is small and unbalanced

cv = StratifiedShuffleSplit(labels, n_iter=100, test_size=0.75, random_state = 42)
for train_index, test_index in cv:
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    for ii in train_index:
        X_train.append( features[ii] )
        y_train.append( labels[ii] )
    for jj in test_index:
        X_test.append( features[jj] )
        y_test.append( labels[jj] )

In [156]:
def classifiers_cv(clf_type,labels_train,features_train,labels_test,features_test):
    
    t0 = time()
    if clf_type == 'NB':
        clf = GaussianNB()
    elif clf_type == 'DT':
        clf = DecisionTreeClassifier()
    elif clf_type == 'KM':
        clf = KMeans(n_clusters=2)
    elif clf_type == 'AB':
        clf = AdaBoostClassifier()
    else:
        print 'Cannot identify classifier'
        
    clf.fit(features_train, labels_train)
    pred = clf.predict(features_test)
    accuracy = accuracy_score(pred,labels_test)
    print "Accuracy: ", accuracy
    print "Algorithm time:", round(time()-t0, 3), "s"

    report = classification_report(labels_test, pred)
    
    print report
    
    
# Try Decision Tree Classifier with cross validation
classifiers_cv('DT',y_train, X_train, y_test,X_test)

Accuracy:  0.851851851852
Algorithm time: 0.002 s
             precision    recall  f1-score   support

        0.0       0.88      0.96      0.92        94
        1.0       0.33      0.14      0.20        14

avg / total       0.81      0.85      0.83       108



In [157]:
# Try Naive Bayes Classifier with cross validation
classifiers_cv('NB',y_train, X_train, y_test,X_test)

Accuracy:  0.851851851852
Algorithm time: 0.004 s
             precision    recall  f1-score   support

        0.0       0.89      0.95      0.92        94
        1.0       0.38      0.21      0.27        14

avg / total       0.82      0.85      0.83       108



In [158]:
# Try AdaBooster Classifier with cross validation
classifiers_cv('AB',y_train, X_train, y_test,X_test)

Accuracy:  0.87962962963
Algorithm time: 0.178 s
             precision    recall  f1-score   support

        0.0       0.94      0.93      0.93        94
        1.0       0.53      0.57      0.55        14

avg / total       0.88      0.88      0.88       108



In [159]:
# Try KMeans Classifier with cross validation
classifiers_cv('KM',y_train, X_train, y_test,X_test)

Accuracy:  0.87037037037
Algorithm time: 0.014 s
             precision    recall  f1-score   support

        0.0       0.87      1.00      0.93        94
        1.0       0.00      0.00      0.00        14

avg / total       0.76      0.87      0.81       108



In [160]:
# Test on Naive Bayes
clf = GaussianNB()
tester.dump_classifier_and_data(clf,my_dataset,features_list)
tester.main();

GaussianNB(priors=None)
	Accuracy: 0.74953	Precision: 0.23642	Recall: 0.39400	F1: 0.29552	F2: 0.34766
	Total predictions: 15000	True positives:  788	False positives: 2545	False negatives: 1212	True negatives: 10455



In [161]:
# Test on Decision Tree Classifier
clf = DecisionTreeClassifier()
tester.dump_classifier_and_data(clf,my_dataset,features_list)
tester.main();

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
	Accuracy: 0.81887	Precision: 0.32452	Recall: 0.33150	F1: 0.32797	F2: 0.33008
	Total predictions: 15000	True positives:  663	False positives: 1380	False negatives: 1337	True negatives: 11620



In [162]:
# Test on KMeans
clf = KMeans(n_clusters = 2)
tester.dump_classifier_and_data(clf,my_dataset,features_list)
tester.main();

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
	Accuracy: 0.83033	Precision: 0.22108	Recall: 0.10800	F1: 0.14511	F2: 0.12031
	Total predictions: 15000	True positives:  216	False positives:  761	False negatives: 1784	True negatives: 12239



In [163]:
# Scale features

#df = pd.DataFrame(data,columns = features_list)
#df.replace('NaN', np.nan, inplace=True)
#print df.shape

scaled_df = df[features_list].copy()
scaled_df.ix[:,1:]=scale(scaled_df.ix[:,1:])
my_dataset = scaled_df.to_dict(orient = 'index')

scaled_df['poi'].value_counts()

False    126
True      18
Name: poi, dtype: int64

In [164]:
# Test on Naive Bayes
clf = GaussianNB()
tester.dump_classifier_and_data(clf,my_dataset,features_list)
tester.main();

GaussianNB(priors=None)
	Accuracy: 0.40847	Precision: 0.16160	Recall: 0.82050	F1: 0.27001	F2: 0.45194
	Total predictions: 15000	True positives: 1641	False positives: 8514	False negatives:  359	True negatives: 4486



In [165]:
# Test on Decision Tree Classifier
clf = DecisionTreeClassifier()
tester.dump_classifier_and_data(clf,my_dataset,features_list)
tester.main();

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
	Accuracy: 0.81993	Precision: 0.32725	Recall: 0.33200	F1: 0.32961	F2: 0.33104
	Total predictions: 15000	True positives:  664	False positives: 1365	False negatives: 1336	True negatives: 11635



In [166]:
# Test on KMeans
clf = KMeans(n_clusters = 2)
tester.dump_classifier_and_data(clf,my_dataset,features_list)
tester.main();

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
	Accuracy: 0.81060	Precision: 0.16333	Recall: 0.10200	F1: 0.12558	F2: 0.11028
	Total predictions: 15000	True positives:  204	False positives: 1045	False negatives: 1796	True negatives: 11955



In [167]:
# Create new features
for v in data_dict_cleaned.values():
    from_poi_to_this_person = v["from_poi_to_this_person"]
    to_messages = v["to_messages"]
    from_this_person_to_poi = v["from_this_person_to_poi"]
    from_messages = v["from_messages"]
    shared_receipt_with_poi = v['shared_receipt_with_poi']
    bonus = v['bonus']
    salary = v['salary']
    total_payments = v['total_payments']

    v["from_poi_ratio"] = (float(from_poi_to_this_person) / float(to_messages) if
                           to_messages not in [0, "NaN"] and from_poi_to_this_person
                           not in [0, "NaN"] else 0.0)
    v["to_poi_ratio"] = (float(from_this_person_to_poi) / float(from_messages) if
                         from_messages not in [0, "NaN"] and from_this_person_to_poi
                         not in [0, "NaN"] else 0.0)
    v["shared_poi_ratio"] = (float(shared_receipt_with_poi) / float(to_messages) if
                         to_messages not in [0, "NaN"] and shared_receipt_with_poi
                         not in [0, "NaN"] else 0.0)
    v["bonus_salary_ratio"] = (float(bonus) / float(salary) if
                         salary not in [0, "NaN"] and bonus
                         not in [0, "NaN"] else 0.0)
    v["bonus_ratio"] = (float(bonus) / float(total_payments) if
                         total_payments not in [0, "NaN"] and bonus
                         not in [0, "NaN"] else 0.0)

features_list.append("from_poi_ratio")
features_list.append("to_poi_ratio")
features_list.append("shared_poi_ratio")
features_list.append("bonus_salary_ratio")
features_list.append("bonus_ratio")
#print features_list
features_list_all = features_list
print features_list_all

['poi', 'bonus', 'salary', 'deferral_payments', 'deferred_income', 'director_fees', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances', 'long_term_incentive', 'other', 'restricted_stock', 'restricted_stock_deferred', 'shared_receipt_with_poi', 'to_messages', 'total_payments', 'total_stock_value', 'from_poi_ratio', 'to_poi_ratio', 'shared_poi_ratio', 'bonus_salary_ratio', 'bonus_ratio']


In [168]:
# Re-orgnize dataset
my_dataset = data_dict_cleaned
data = featureFormat(my_dataset, features_list)
labels, features = targetFeatureSplit(data)

In [169]:
# Test on Naive Bayes
clf = GaussianNB()
tester.dump_classifier_and_data(clf,my_dataset,features_list)
tester.main();

GaussianNB(priors=None)
	Accuracy: 0.74953	Precision: 0.23642	Recall: 0.39400	F1: 0.29552	F2: 0.34766
	Total predictions: 15000	True positives:  788	False positives: 2545	False negatives: 1212	True negatives: 10455



In [170]:
# Test on Decision Tree Classifier
clf = DecisionTreeClassifier()
tester.dump_classifier_and_data(clf,my_dataset,features_list)
tester.main();

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
	Accuracy: 0.89727	Precision: 0.61446	Recall: 0.61600	F1: 0.61523	F2: 0.61569
	Total predictions: 15000	True positives: 1232	False positives:  773	False negatives:  768	True negatives: 12227



In [171]:
# Feature selection

# Get feature importance of the DecisionTree Classifier
clf.fit(features, labels)
tree_feature_importances = clf.feature_importances_
tree_features = zip(tree_feature_importances, features_list[1:])
tree_features = sorted(tree_features, key = lambda x:x[0], reverse = True)

print('Decision Tree Feature Importance: \n')
for i in range(10):
    print ('{}:{:.4f}'.format(tree_features[i][1], tree_features[i][0]))

Decision Tree Feature Importance: 

to_poi_ratio:0.3469
shared_receipt_with_poi:0.2675
expenses:0.1680
other:0.1563
from_poi_ratio:0.0612
bonus:0.0000
salary:0.0000
deferral_payments:0.0000
deferred_income:0.0000
director_fees:0.0000


In [176]:
# Use GridSearchCV and SelectKBest to find the optimal number of features

n_features = np.arange(1,len(features_list))

pipe = Pipeline([
        ('select_features',SelectKBest()),
        ('classify',DecisionTreeClassifier())
    ])

param_grid = [
    {'select_features__k': n_features}
]

tree_clf = GridSearchCV(pipe, param_grid = param_grid, scoring = 'f1', cv = 10)
tree_clf.fit(features, labels)
print tree_clf.best_params_

{'select_features__k': 20}


In [177]:
print tree_clf.best_estimator_

Pipeline(memory=None,
     steps=[('select_features', SelectKBest(k=20, score_func=<function f_classif at 0x10f4d1050>)), ('classify', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])


In [178]:
scores = [x[1] for x in tree_clf.grid_scores_]
features_scores = [x[0] for x in tree_clf.grid_scores_]
scores = np.array(scores).reshape(len(n_features))

scores_zip = zip(scores, features_list_all[1:])
scores_sorted = sorted(scores_zip, key = lambda x:x[0], reverse = True)
#print scores_sorted

features_sorted = []
features_sorted.append('poi')
print('Scores: \n')
for i in range(len(n_features)):
    print ('{}:{:.4f}'.format(scores_sorted[i][1], scores_sorted[i][0]))
    features_sorted.append(scores_sorted[i][1])

Scores: 

from_poi_ratio:0.6771
to_poi_ratio:0.6771
total_payments:0.6644
bonus_ratio:0.6644
bonus_salary_ratio:0.6470
restricted_stock:0.6042
total_stock_value:0.5949
shared_poi_ratio:0.5938
to_messages:0.5729
restricted_stock_deferred:0.5671
shared_receipt_with_poi:0.5451
other:0.4873
long_term_incentive:0.4097
loan_advances:0.3819
director_fees:0.3127
deferral_payments:0.2824
expenses:0.2812
from_messages:0.2049
from_this_person_to_poi:0.1933
from_poi_to_this_person:0.1910
salary:0.1817
deferred_income:0.1343
bonus:0.1299
exercised_stock_options:0.1169


In [179]:
print features_sorted[:22]

['poi', 'from_poi_ratio', 'to_poi_ratio', 'total_payments', 'bonus_ratio', 'bonus_salary_ratio', 'restricted_stock', 'total_stock_value', 'shared_poi_ratio', 'to_messages', 'restricted_stock_deferred', 'shared_receipt_with_poi', 'other', 'long_term_incentive', 'loan_advances', 'director_fees', 'deferral_payments', 'expenses', 'from_messages', 'from_this_person_to_poi', 'from_poi_to_this_person', 'salary']


In [180]:
features_list = features_sorted[:22]

print len(features_list)
clf = DecisionTreeClassifier()
tester.dump_classifier_and_data(clf,my_dataset,features_list)
tester.main();

22
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
	Accuracy: 0.89760	Precision: 0.61406	Recall: 0.62450	F1: 0.61924	F2: 0.62238
	Total predictions: 15000	True positives: 1249	False positives:  785	False negatives:  751	True negatives: 12215



In [135]:
features_list = []
features_list.append("poi")
features_list.append("expenses")
features_list.append("other")
features_list.append("exercised_stock_options")
features_list.append("to_poi_ratio")
features_list.append("shared_receipt_with_poi")

    
print features_list

clf = DecisionTreeClassifier()
tester.dump_classifier_and_data(clf,my_dataset,features_list)
tester.main();

['poi', 'expenses', 'other', 'exercised_stock_options', 'to_poi_ratio', 'shared_receipt_with_poi']
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
	Accuracy: 0.91727	Precision: 0.69572	Recall: 0.67450	F1: 0.68495	F2: 0.67864
	Total predictions: 15000	True positives: 1349	False positives:  590	False negatives:  651	True negatives: 12410



In [181]:
features_list = []
features_list.append("poi")
features_list.append("expenses")
features_list.append("other")
features_list.append("from_poi_ratio")
features_list.append("to_poi_ratio")
features_list.append("shared_receipt_with_poi")

    
print features_list

clf = DecisionTreeClassifier()
tester.dump_classifier_and_data(clf,my_dataset,features_list)
tester.main();

['poi', 'expenses', 'other', 'from_poi_ratio', 'to_poi_ratio', 'shared_receipt_with_poi']
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
	Accuracy: 0.91447	Precision: 0.67582	Recall: 0.68900	F1: 0.68235	F2: 0.68632
	Total predictions: 15000	True positives: 1378	False positives:  661	False negatives:  622	True negatives: 12339



In [182]:
features_list = []
features_list.append("poi")
features_list.append("deferred_income")
features_list.append("exercised_stock_options")
features_list.append("director_fees")
features_list.append("to_poi_ratio")
features_list.append("bonus_salary_ratio")

print features_list

clf = DecisionTreeClassifier()
tester.dump_classifier_and_data(clf,my_dataset,features_list)
tester.main();

['poi', 'deferred_income', 'exercised_stock_options', 'director_fees', 'to_poi_ratio', 'bonus_salary_ratio']
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
	Accuracy: 0.85607	Precision: 0.45065	Recall: 0.36300	F1: 0.40210	F2: 0.37769
	Total predictions: 15000	True positives:  726	False positives:  885	False negatives: 1274	True negatives: 12115



In [183]:
features_list = features_sorted[:22]

my_dataset = data_dict_cleaned
data = featureFormat(my_dataset, features_list)
labels, features = targetFeatureSplit(data)

tree_pipe = Pipeline([
        ('select_features',SelectKBest(k=20)),
        ('classify',DecisionTreeClassifier())
    ]
)

param_grid = {'classify__criterion' : ('gini','entropy'),
              'classify__splitter' : ('best','random'),
              'classify__max_depth' : (None, 5, 10, 15, 20),
              'classify__min_samples_split' : (2,4,6,8,10,20),
              'classify__max_features' : (None, 'sqrt', 'auto', 'log2')}


tree_clf = GridSearchCV(tree_pipe, param_grid, scoring = 'recall', cv = 10)
tree_clf.fit(features, labels)
print tree_clf.best_params_

{'classify__max_features': None, 'classify__splitter': 'best', 'classify__min_samples_split': 4, 'classify__criterion': 'entropy', 'classify__max_depth': None}


In [185]:
tree_clf = Pipeline([
        ('select_features',SelectKBest(k=20)),
        ('classify',DecisionTreeClassifier(max_features = None, splitter='best', min_samples_split=4, criterion='entropy',
                                          max_depth = None))
    ]
)


#tree_clf = DecisionTreeClassifier()
tester.dump_classifier_and_data(tree_clf,my_dataset,features_list)
tester.main();

Pipeline(memory=None,
     steps=[('select_features', SelectKBest(k=20, score_func=<function f_classif at 0x10f4d1050>)), ('classify', DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])
	Accuracy: 0.90580	Precision: 0.63930	Recall: 0.67350	F1: 0.65595	F2: 0.66637
	Total predictions: 15000	True positives: 1347	False positives:  760	False negatives:  653	True negatives: 12240

